* Data from https://opendata.dwd.de/weather/crowdsourcing/warnwetter/opendata_wettermeldung_20230622.geojson.bz2

In [1]:
using GeoJSON

In [103]:
warnwetter=GeoJSON.read("opendata_wettermeldung_20230622.geojson")

FeatureCollection with 6509 Features

In [3]:
using DataFrames

In [104]:
warnwetter_df=DataFrame(warnwetter)


Row,timestamp,zusatzAttributeWithValues,geometry,place,meldungId,zusatzAttribute,category,auspraegung
,Int64,Array…,Point…,String?,Int64,Array…,String,String
1,1693008008451,Any[],2D Point,Stutensee,4600252,Any[],REGEN,REGEN_STARK
2,1693008009124,Any[],2D Point,Meiningen,4600253,Any[],HAGEL,HAGEL_2CM
3,1693008017537,Any[],2D Point,Karlsruhe,4600254,Any[],REGEN,REGEN_MITTEL
4,1693008023986,Any[],2D Point,Neuberg,4600255,Any[],REGEN,REGEN_EXTREM
5,1693008030796,Any[],2D Point,Karlsruhe,4600273,Any[],REGEN,REGEN_EXTREM
6,1693008047285,Any[],2D Point,Schotten,4600256,Any[],REGEN,REGEN_KRAEFTIG
7,1693008053642,Any[],2D Point,Karlsdorf-Neuthard,4600321,Any[],BLITZE,BLITZE_EXTREM
8,1693008060318,Any[],2D Point,Meiningen,4600257,Any[],WIND,WIND_STUERMISCH
9,1693008065141,Any[],2D Point,Neuengronau,4600259,Any[],REGEN,REGEN_STARK


In [7]:
using Dates

In [108]:
warnwetter_df.datetime.=unix2datetime.(warnwetter_df.timestamp./1000);
warnwetter_df.lons.=first.(warnwetter_df.geometry)
warnwetter_df.lats.=last.(warnwetter_df.geometry)
warnwetter_df

Row,timestamp,zusatzAttributeWithValues,geometry,place,meldungId,zusatzAttribute,category,auspraegung,datetime,lons,lats
,Int64,Array…,Point…,String?,Int64,Array…,String,String,DateTime,Float32,Float32
1,1687392279207,Any[],2D Point,Mömlingen,4138360,Any[],WIND,WIND_STARK,2023-06-22T00:04:39.207,9.085,49.855
2,1687392338842,Any[],2D Point,Rodgau,4138361,Any[],REGEN,REGEN_LEICHT,2023-06-22T00:05:38.842,8.865,49.995
3,1687392385812,Any[],2D Point,Heiligkreuz,4138362,Any[],REGEN,REGEN_LEICHT,2023-06-22T00:06:25.812,6.645,49.74
4,1687392446502,Any[],2D Point,Homburg,4138363,Any[],REGEN,REGEN_LEICHT,2023-06-22T00:07:26.502,7.335,49.305
5,1687392553066,Any[],2D Point,Rastatt,4138366,Any[],WIND,WIND_SCHWACH,2023-06-22T00:09:13.066,8.22,48.865
6,1687392718515,Any[],2D Point,Kördorf,4138367,Any[],REGEN,REGEN_LEICHT,2023-06-22T00:11:58.515,7.92,50.285
7,1687392723000,Any[],2D Point,Bietigheim,4138369,Any[],BLITZE,BLITZE_EINZELNE,2023-06-22T00:12:03,8.25,48.91
8,1687392819849,Any[],2D Point,Rastatt,4138368,Any[],REGEN,REGEN_KRAEFTIG,2023-06-22T00:13:39.849,8.22,48.865
9,1687393018036,Any[],2D Point,Rettershain,4138370,Any[],REGEN,REGEN_LEICHT,2023-06-22T00:16:58.036,7.83,50.145


In [24]:
using GLMakie

In [96]:
using ArchGDAL

In [97]:
layer=ArchGDAL.getlayer(ArchGDAL.read("../nuts5000_12-31.gk3.shape/nuts5000/NUTS5000_N1.shp"),0)

Layer: NUTS5000_N1
  Geometry 0 (): [wkbPolygon], MULTIPOLYGON (((3477...), ...
     Field 0 (OBJID): [OFTString], DEBKGNU5000000C8, DEBKGNU5000000C9, ...
     Field 1 (BEGINN): [OFTDate], 2021-10-04T00:00:00, 2021-10-04T00:00:00, ...
     Field 2 (GF): [OFTInteger], 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, ...
     Field 3 (NUTS_LEVEL): [OFTInteger], 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
     Field 4 (NUTS_CODE): [OFTString], DE1, DE2, DE3, DE4, DE5, DE6, DE7, ...
...
 Number of Fields: 6

In [98]:
bounds=DataFrame(layer);

In [99]:
ArchGDAL.createcoordtrans(ArchGDAL.importURL("https://sg.geodatenzentrum.de/web_public/gdz/dokumentation/crs/gk3.prj"), ArchGDAL.importPROJ4("+proj=longlat +datum=WGS84 +no_defs +type=crs")) do transform
    for x in eachrow(bounds)
    ArchGDAL.transform!(x."", transform)
    end
end

In [94]:
using GeoMakie

In [167]:
einwohner=DataFrame(GeoJSON.read("../Kreisgrenzen_2021_mit_Einwohnerzahl.geojson"))

Row,geometry,OBJECTID,BEM,BEGINN,FK_S3,KFL,BEZ,SHAPE_Length,NUTS,IBZ,SN_V1,EWZ,GEN,SN_L,SHAPE_Area,WSK,AGS,SN_R,OBJID,SN_G,SN_K,SN_V2
,MultiPol…,Int64,String,String,String,Union…,String,Float64,String,Int64,String,Int64,String,String,Float64,String,String,String,String,String,String,String
1,2D MultiPolygon,1,--,2021-12-02T00:00:00Z,R,56.73,Kreisfreie Stadt,0.527766,DEF01,40,00,91113,Flensburg,01,0.00687941,2008-01-01T00:00:00Z,01001,0,DEBKGVG200000C75,000,01,00
2,2D MultiPolygon,2,--,2021-11-30T00:00:00Z,R,118.65,Kreisfreie Stadt,1.27388,DEF02,40,00,246243,Kiel,01,0.0155083,2006-01-01T00:00:00Z,01002,0,DEBKGVG200000C76,000,02,00
3,2D MultiPolygon,3,--,2021-06-24T00:00:00Z,R,214.19,Kreisfreie Stadt,1.83544,DEF03,40,00,216277,Lübeck,01,0.0289147,2006-02-01T00:00:00Z,01003,0,DEBKGVG200000C77,000,03,00
4,2D MultiPolygon,4,--,2021-08-24T00:00:00Z,R,71.66,Kreisfreie Stadt,0.65486,DEF04,40,00,79496,Neumünster,01,0.00983232,1970-04-26T00:00:00Z,01004,0,DEBKGVG200000C78,000,04,00
5,2D MultiPolygon,5,--,2021-11-17T00:00:00Z,R,1428.18,Kreis,3.08157,DEF05,42,00,133969,Dithmarschen,01,0.196341,2011-08-01T00:00:00Z,01051,0,DEBKGVG200000C79,000,51,00
6,2D MultiPolygon,6,--,2021-12-01T00:00:00Z,R,1263.06,Kreis,3.17723,DEF06,42,00,200819,Herzogtum Lauenburg,01,0.171358,2009-01-01T00:00:00Z,01053,0,DEBKGVG200000C7A,000,53,00
7,2D MultiPolygon,7,--,2021-12-06T00:00:00Z,R,2083.53,Kreis,7.99934,DEF07,42,00,167560,Nordfriesland,01,0.290658,2016-01-01T00:00:00Z,01054,0,DEBKGVG200000C7B,000,54,00
8,2D MultiPolygon,8,--,2022-01-04T00:00:00Z,R,1393,Kreis,4.48118,DEF08,42,00,202014,Ostholstein,01,0.190837,2010-01-01T00:00:00Z,01055,0,DEBKGVG200000C7C,000,55,00
9,2D MultiPolygon,9,--,2021-10-25T00:00:00Z,R,664.27,Kreis,2.22135,DEF09,42,00,318326,Pinneberg,01,0.0868169,2009-01-01T00:00:00Z,01056,0,DEBKGVG200000C7D,000,56,00


In [209]:
cmap=:YlOrBr_9
fig=Figure(backgroundcolor=:black,resolution=(1000,800))
Label(fig[1,1:2],text="Number of observations are similar to the population\nBased on the WarnWetter-App data from 22.06.2023",color=:white,fontsize=30)
ax=Axis(fig[2,1],limits=(5,15,47,55),backgroundcolor=:black,width=500)
a=hexbin!(ax,warnwetter_df.lons,warnwetter_df.lats,cellsize=0.06,colorrange=(0,40),threshold=1,colormap=cgrad(cmap,rev=true), highclip=cgrad(cmap)[0.0],strokewidth = 0.5,strokecolor = :gray50)
poly!(ax,GeoMakie.to_multipoly.(GeoMakie.geo2basic.(einwohner.geometry)),color=:transparent,strokecolor=:white,strokewidth=0.05)

ax=Axis(fig[2,2],limits=(5,15,47,55),backgroundcolor=:black,width=500)

b=poly!(ax,GeoMakie.to_multipoly.(GeoMakie.geo2basic.(einwohner.geometry)),color=einwohner.EWZ,colorrange=(10000,700000),colormap=cgrad(cmap,rev=true),lowclip=cgrad(cmap)[1.0], highclip=cgrad(cmap)[0.0])
Colorbar(fig[3,2],b,vertical=false,label="Population in county",labelcolor=:white,tickcolor=:white,ticklabelcolor=:white)

#poly!(g,GeoMakie.to_multipoly.(GeoMakie.geo2basic.(bounds."")),color=:transparent,strokecolor=:white,strokewidth=1)
Colorbar(fig[3,1],a,vertical=false,label="Number of observations in hex",labelcolor=:white,tickcolor=:white,ticklabelcolor=:white)
Label(fig[4,1],text="Data: © Deutscher Wetterdienst\nGeodata: © GeoBasis-DE / BKG (2023) and  © GeoBasis-DE / BKG, Statistisches Bundesamt (Destatis) (2022)",justification=:left,color=:white)
Makie.resize_to_layout!(fig)
fig

In [210]:
save("hexagons.png",fig)